In [1]:
trace_directory = '/tmp/session-pingpong/ust'

In [2]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/ros2/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../ros2/ros2_tracing/tracetools_read/')
import os

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
import numpy as np
import pandas as pd

from tracetools_analysis.analysis import load
from tracetools_analysis.analysis import ros2_processor
from tracetools_analysis.analysis import utils
from tracetools_analysis.conversion import ctf

In [3]:
# Convert
pickle_filename = 'pickle'
pickle_path = os.path.join(trace_directory, pickle_filename)
count = ctf.convert(trace_directory, pickle_path)
print(f'{count} events')

# Process
events = load.load_pickle(pickle_path)
processor = ros2_processor.ros2_process(events)
data_model = processor.get_data_model()
#data_model.print_model()

374 events


In [4]:
data_util = utils.DataModelUtil(data_model)

callback_symbols = data_util.get_callback_symbols()

output_notebook()
psize = 450

# Plot durations
for obj, symbol in callback_symbols.items():
    owner_info = data_util.get_callback_owner_info(obj)
    if owner_info is None:
        owner_info = '[unknown]'

    # Duration
    duration_ns = data_util.get_callback_durations(obj)
    duration_ms = duration_ns.apply(lambda d: d/1000000.0)
    source = ColumnDataSource(duration_ms)
    duration = figure(title=owner_info, x_axis_label='start', y_axis_label='duration (ms)',
                      plot_width=psize, plot_height=psize)
    duration.title.align = 'center'
    duration.line(x='timestamp', y='duration', legend=str(symbol), line_width=2, source=source)

    # Histogram
    dur_hist, edges = np.histogram(duration_ms['duration'])
    duration_ms_hist = pd.DataFrame({'duration': dur_hist, 
                                     'left': edges[:-1], 
                                     'right': edges[1:]})
    hist = figure(title='Duration histogram', x_axis_label='duration (ms)', y_axis_label='frequency',
                  plot_width=psize, plot_height=psize)
    hist.title.align = 'center'
    hist.quad(bottom=0, top=duration_ms_hist['duration'], 
              left=duration_ms_hist['left'], right=duration_ms_hist['right'])

    show(row(duration, hist))

Loading BokehJS ...